### Choose the first entry of each patient. Extract heart rate data into ***HR***, create ***HR_index*** for each patient. 
Contains patient heart rate data from both vitalPeriodic.csv & nurseCharting.csv

**1. Extract sub-categories patient id from cardiovascular**

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import data_toolbox

# change to your folder path
#os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/DataExtract/data') 
os.chdir('/home/en580-zxia028/EICU/data')

# import diagnosis.csv
df_diagnosis = pd.read_csv('diagnosis.csv')
df_diagnosis.sort_values(by=['patientunitstayid', 'diagnosisoffset'], inplace=True)

# select cardiovascular patients
df_cardiovascular = df_diagnosis[df_diagnosis['diagnosisstring'].str.contains('cardiovascular')]
# print(df_cardiovascular)


# get shock patient 
shock_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('shock')]
# print(shock_patient)

# get ventricular patient 
ventricular_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('ventricular')]
# print(ventricular_patient)

# get chest pain patient 
chest_pain_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('chest pain')]

# get arrhythmias patient 
arrhythmias_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('arrhythmias')]

# put id together
df_wanted = pd.concat([shock_patient, ventricular_patient, chest_pain_patient, arrhythmias_patient])
# print(df_wanted)

# Get the patient ids from df_wanted & sort the patient id
# patient_id_all multiple entry patient's stayid
patient_id_all = df_wanted['patientunitstayid'].unique()
patient_id_all.sort()
print(patient_id_all)

[ 141168  141203  141227 ... 3353216 3353235 3353251]


**2. Exclude patient whose unitvisitnumbe>1**

In [2]:
# import patient.csv
df_patient = pd.read_csv('patient.csv')
df_patient.sort_values(by=['patientunitstayid'], inplace=True)
df_patient_buf = df_patient[df_patient['patientunitstayid'].isin(patient_id_all)]
df_1time_patient = df_patient_buf[df_patient_buf['unitvisitnumber']==1]
# print(df_1time_patient)

# select the patient id from df_1time_patient
patient_id = df_1time_patient['patientunitstayid'].unique()
print(f'Total number of patients: {len(patient_id)}')

Total number of patients: 71353


**3. Extract data of patients within the id list**

In [3]:
# import vitalPeriodic.csv & nurseCharting.csv
df_vitalPeriodic = pd.read_csv('vitalPeriodic.csv')
df_vitalPeriodic.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)


# select the wanted patient
df_vitalPeriodic = df_vitalPeriodic[df_vitalPeriodic['patientunitstayid'].isin(patient_id)]
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# save the wanted file (uncomment the code to save)
# df_vitalPeriodic.to_csv('vitalPeriodic_wanted.csv', index=False)

# print the shape of the wanted file
print(f'vitalperiodic shape: {df_vitalPeriodic.shape}')
print(f'nurseCharting shape: {df_nurseCharting.shape}')

vitalperiodic shape: (63195275, 19)
nurseCharting shape: (61929777, 8)


**4. Extract Heart Rate data & create index**

It is weird that some patient id doesn't exist in df_vitalPeriodic, maybe it's a demo problem

In [4]:
# value1 = set(df_vitalPeriodic['patientunitstayid'].unique())
# value2 = set(patient_id)
# unique_to_2 = value2.difference(value1)
# print(unique_to_2)

In [5]:
# define heartrate preprocessing function
def normal_heartrate(num):
    """
    Function to normalize heart rate values.

    Parameters:
        num: the originial input value
    Return:
        num: the normalized output value
    """
    # Return null values direcly
    if pd.isna(num):
        return num
    # Remove values out of range
    elif num > 300 or num < 0:
        return np.nan
    # Return normal values directly
    else:
        return num

In [6]:
# nursingchartcelltypevallabel Heart Rate
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Heart Rate']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'heartrate'})
print(df_nurseCharting.head())

           nursingchartid  patientunitstayid  observationoffset  \
151470706       164546008             141168                  6   
151470614       146409413             141168                 21   
151470622       146455198             141168                 36   
151470649       201018075             141168                 51   
151470709       200253204             141168                 66   

           nursingchartentryoffset nursingchartcelltypecat  \
151470706                        6             Vital Signs   
151470614                       21             Vital Signs   
151470622                       36             Vital Signs   
151470649                       51             Vital Signs   
151470709                       66             Vital Signs   

          nursingchartcelltypevallabel nursingchartcelltypevalname heartrate  
151470706                   Heart Rate                  Heart Rate       140  
151470614                   Heart Rate                  Heart Rate

In [9]:
# extract heart rate from df_vitalPeriodic & df_nurseCharting
HR_v = df_vitalPeriodic[['patientunitstayid', 'observationoffset', 'heartrate']]
HR_n = df_nurseCharting[['patientunitstayid', 'observationoffset', 'heartrate']]
HR = pd.concat([HR_v, HR_n]).astype(float)
HR.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of HR: \n{HR.head()} \n')

# exclude abnormal heart rate values
HR.loc[:, 'heartrate'] = HR['heartrate'].apply(normal_heartrate)

# save HR to csv file (uncomment the code to save)
# HR.to_csv('HR.csv', index=False)

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(HR['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(HR))
# create first occurrence index for every patient
HR_index = pd.Series(first_occurrences)
print(f'shape of HR_index: {HR_index.shape}')
print(f'First 5 rows of HR_index: \n{HR_index.head()}')

# double check the index is correct
# print(HR.iloc[HR_index].head())
# print(HR.iloc[[156, 157, 158, 159]])
# print(HR.iloc[[1015, 1016, 1017, 1018]])

First 5 rows of HR: 
           patientunitstayid  observationoffset  heartrate
151470706           141168.0                6.0      140.0
151470614           141168.0               21.0       70.0
151470622           141168.0               36.0       70.0
151470649           141168.0               51.0      140.0
151470709           141168.0               66.0      140.0 

shape of HR_index: (71243,)
First 5 rows of HR_index: 
0       0
1     468
2     977
3    1423
4    1475
dtype: int64


**Example: how to use HR & HR_index**

In [8]:
# if we want the i th patient's data (i starts from 0)
# use HR.iloc[HR_index[i]:HR_index[i+1]]
i = 0
print(f'HeartRate data for patient {i+1}: \n{HR.iloc[HR_index[i]:HR_index[i+1]]}')

HeartRate data for patient 1: 
           patientunitstayid  observationoffset  heartrate
151470706           141168.0                6.0      140.0
151470614           141168.0               21.0       70.0
151470622           141168.0               36.0       70.0
151470649           141168.0               51.0      140.0
151470709           141168.0               66.0      140.0
...                      ...                ...        ...
371                 141168.0             2294.0      100.0
407                 141168.0             2299.0      100.0
125                 141168.0             2304.0      100.0
101                 141168.0             2309.0      100.0
203                 141168.0             2314.0      100.0

[468 rows x 3 columns]
